In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
df = pd.read_csv("EURUSD_Candlestick_1_Hour_BID_09.05.2022-14.05.2022.csv")
    #"USA30.IDXUSD_Candlestick_1_Hour_BID_01.01.2020-09.04.2022.csv")
#"GBPUSD_Candlestick_1_Hour_BID_04.01.2015-09.04.2022.csv"
#"GBPUSD_Candlestick_1_Hour_BID_01.01.2022-09.04.2022.csv")
#"GBPUSD_Candlestick_1_Hour_BID_01.01.2010-01.01.2021.csv")

In [3]:
df = df.iloc[:-23]

df['datetime'] = "NA"
df['datetime'] = pd.date_range(start='05/09/2022', end='05/14/2022', freq='1H')#start='01/01/2010', end='01/02/2021', freq='1H')
df['weekday'] = df['datetime'].dt.day_name()
df['hour'] = df['datetime'].dt.hour
df['week'] = df['datetime'].dt.week
df.set_index('datetime', inplace=True)

#this doesnt mimic conventional charts tho booooo would need to shift starting point (Or just use data in NY time)
df['WeekNum'] = np.divmod(np.arange(len(df)),168)[0]+1

/tmp/ipykernel_76042/1298862182.py:7: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['week'] = df['datetime'].dt.week


In [4]:
#filter out the NaN values and anything that is 0 volume (Weekends & Holidays)
indexZeroV = df[ df['Volume'] == 0].index
df.drop(indexZeroV, inplace = True)

#create df for bull weeks and bear weeks
#group by week number
weekly_grp = df.groupby('WeekNum')

dfW = pd.DataFrame()

dfW['wHigh'] = df.groupby('WeekNum')['High'].max()
dfW['wLow'] = df.groupby('WeekNum')['Low'].min()
dfW['wOpen'] = df.groupby('WeekNum')['Open'].first()
dfW['wClose'] = df.groupby('WeekNum')['Close'].last()

df.isna().sum()

Localtime    0
Open         0
High         0
Low          0
Close        0
Volume       0
weekday      0
hour         0
week         0
WeekNum      0
dtype: int64

In [5]:
dfW['OC'] = dfW['wOpen'] - dfW['wClose']
#take OC of each weeknum then place it in row with that weeknum in df
bins = [np.NINF,0,np.Inf]
state_groups = ["Bull","Bear"]
dfW['Outcome'] = pd.cut(dfW.OC,bins,labels = state_groups)
#duplicate rows
dff = pd.DataFrame()
dff = dfW.loc[dfW.index.repeat(168)]
#move Outcome column to original df
df_dropped_last_n = dff.iloc[:-143]
df_dropped_last_n = df_dropped_last_n.reset_index()
#del df['Localtime']
del df_dropped_last_n['WeekNum']
df = df.reset_index().join(df_dropped_last_n)
df.set_index('datetime', inplace=True)


In [6]:
#So we want to join highs of the week for bear outcomes and lows of the week datetime ids for bull outcomes
weekly_grp_bull = df.groupby('WeekNum')['Low'].idxmin()
weekly_grp_bear = df.groupby('WeekNum')['High'].idxmax()

dfidxmin = pd.DataFrame()
dfidxmin = weekly_grp_bull.loc[weekly_grp_bull.index.repeat(168)]
#move Outcome column to original df
dfidx_min_dropped = dfidxmin.iloc[:-143]
dfidx_min_dropped = dfidx_min_dropped.reset_index()

dfidxmax = pd.DataFrame()
dfidxmax = weekly_grp_bear.loc[weekly_grp_bear.index.repeat(168)]
#move Outcome column to original df
dfidx_max_dropped = dfidxmax.iloc[:-143]
dfidx_max_dropped = dfidx_max_dropped.reset_index()

#merge idx for highs and lows of the week...
df = df.reset_index()
df['WH_ID'] = dfidx_max_dropped['High']
df['WL_ID'] = dfidx_min_dropped['Low']
df.set_index('datetime', inplace=True)

#Classify each row as datetime appriate for the outcome week
#Now we are gonna group by each week and then group that group by bull and bear outcome weeks
bins_classify = ["Bull","Bear"]
states_datetime = [np.NINF,0,np.Inf]

#dfW['Outcome'] = pd.cut(dfW.OC,bins,labels = state_groups)


#df_grped = df.groupby(['WeekNum'], as_index = False)
#After that we will run statistics regarding what day of the week is HOW/LOW what % of the time...


In [11]:
#Create the chosen row that gives the ID of the corresponding high/low of week
#df['Chosen'] = "NA"
df.loc[df.Outcome == "Bear", 'Chosen'] = df['WH_ID']
df.loc[df.Outcome == "Bull", 'Chosen'] = df['WL_ID']
#df['ChosenOpp'] = "NA"
df.loc[df.Outcome == "Bear", 'ChosenOpp'] = df['WL_ID']
df.loc[df.Outcome == "Bull", 'ChosenOpp'] = df['WH_ID']

df

,Localtime,Open,High,Low,Close,Volume,weekday,hour,week,WeekNum,wHigh,wLow,wOpen,wClose,OC,Outcome,WH_ID,WL_ID,Chosen,ChosenOpp
datetime,,,,,,,,,,,,,,,,,,,,
2022-05-09 00:00:00,09.05.2022 00:00:00.000 GMT-0500,1.05122,1.05122,1.05029,1.05074,8521.13,Monday,0,19,1,1.05924,1.03496,1.05122,1.04116,0.01006,Bear,2022-05-09 11:00:00,2022-05-13 08:00:00,2022-05-09 11:00:00,2022-05-13 08:00:00
2022-05-09 01:00:00,09.05.2022 01:00:00.000 GMT-0500,1.05075,1.05142,1.04951,1.05026,10340.81,Monday,1,19,1,1.05924,1.03496,1.05122,1.04116,0.01006,Bear,2022-05-09 11:00:00,2022-05-13 08:00:00,2022-05-09 11:00:00,2022-05-13 08:00:00
2022-05-09 02:00:00,09.05.2022 02:00:00.000 GMT-0500,1.05026,1.05293,1.04977,1.05118,20439.25,Monday,2,19,1,1.05924,1.03496,1.05122,1.04116,0.01006,Bear,2022-05-09 11:00:00,2022-05-13 08:00:00,2022-05-09 11:00:00,2022-05-13 08:00:00
2022-05-09 03:00:00,09.05.2022 03:00:00.000 GMT-0500,1.05116,1.05200,1.05010,1.05072,14745.92,Monday,3,19,1,1.05924,1.03496,1.05122,1.04116,0.01006,Bear,2022-05-09 11:00:00,2022-05-13 08:00:00,2022-05-09 11:00:00,2022-05-13 08:00:00
2022-05-09 04:00:00,09.05.2022 04:00:00.000 GMT-0500,1.05072,1.05303,1.05047,1.05279,9937.17,Monday,4,19,1,1.05924,1.03496,1.05122,1.04116,0.01006,Bear,2022-05-09 11:00:00,2022-05-13 08:00:00,2022-05-09 11:00:00,2022-05-13 08:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-13 11:00:00,13.05.2022 11:00:00.000 GMT-0500,1.04113,1.04152,1.04057,1.04086,4568.31,Friday,11,19,1,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT
2022-05-13 12:00:00,13.05.2022 12:00:00.000 GMT-0500,1.04086,1.04159,1.03971,1.03984,3623.81,Friday,12,19,1,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT
2022-05-13 13:00:00,13.05.2022 13:00:00.000 GMT-0500,1.03985,1.04053,1.03936,1.04048,5202.62,Friday,13,19,1,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT


In [8]:
#Now look up the ID given from the Chosen column and get its day of the week, make that swing day col
#df['swingDOW'] = df.loc[df['Chosen']]
#df['swingDOW'] = 
moo = pd.DataFrame()
temp = pd.DataFrame()
hr = pd.DataFrame()


moo['ans'] = df.loc[df['Chosen']]['weekday']
temp['ansOpp'] = df.loc[df['ChosenOpp']]['weekday']
hr['myHR'] = df.loc[df['Chosen']]['hour']


moo = moo.reset_index()
del moo['datetime']

temp = temp.reset_index()
del temp['datetime']

hr = hr.reset_index()
del hr['datetime']

df = df.reset_index().join(moo)
df = df.reset_index().join(temp)
df = df.reset_index().join(hr)

del df['index']
df.set_index('datetime', inplace=True)



KeyError: '[NaT] not in index'

In [ ]:
#now we want to get the statistics associated with the ans column
#Do some quality checks
#remove saturdays from list
df = df.drop(index=df[df['weekday'] == 'Saturday'].index) #doesnt work?


In [ ]:
df['ans'].value_counts(normalize=True)

In [ ]:
df['ansOpp'].value_counts(normalize=True)

In [ ]:
#df.to_csv('file_done_newx.csv')
#lets try it on newer data that we can actually verify 
df['myHR'].value_counts(normalize=True)